In [ ]:
import pandas as pd
import joblib
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score
from pathlib import Path

# Set the checkpoint directory.
CKPT_DIR = 'model_params_big_test'
# CKPT_DIR = 'res_f1'
# CKPT_DIR = 'res_balanced_accuracy'

# Paths to the saved model and scaler.
xgb_model_path = Path(CKPT_DIR) / 'xgboost_model.json'
scaler_path = Path(CKPT_DIR) / 'std_scaler.bin'
df = pd.read_csv(Path(CKPT_DIR) / 'data.csv')

df.columns.tolist()

# Load the scaler and the pre-trained XGBoost model.
scaler = joblib.load(scaler_path)
model = xgb.XGBClassifier()
model.load_model(xgb_model_path)

# Load the dataset that contains ground truth labels.

# Separate the true labels from the dataset.
y_true = df['label']

# Define the columns that were not used as features during training.
columns_to_drop = [
    'sku_first', 'sku_second',
    'name_first', 'description_first',
    'name_second', 'description_second',
    'options_first', 'options_second',
    'image_url_first', 'image_url_second',
    'label'
]

# Create a DataFrame for scaling by dropping the extra columns.
# The original df remains unchanged.
X = df.drop(columns=columns_to_drop, errors='ignore')

# Ensure the columns match exactly what the scaler was trained on.
# The scaler's attribute 'feature_names_in_' holds the expected column names.
if hasattr(scaler, 'feature_names_in_'):
    expected_features = list(scaler.feature_names_in_)
    X_for_scaler = df[expected_features]
else:
    X_for_scaler = df

# print("Columns used for scaling:", X_for_scaler.columns.tolist())

# Scale the features using the loaded scaler.
X_scaled = scaler.transform(X_for_scaler)

# Run inference using the pre-trained model.
predictions = model.predict(X_scaled)

# Calculate metrics.
accuracy = accuracy_score(y_true, predictions)
f1 = f1_score(y_true, predictions, average='weighted')

# print("Predicted classes:", predictions)
print("Accuracy:", accuracy)
print("F1-score:", f1)

Accuracy: 0.753719340570969
F1-score: 0.7431081339773574
